In [1]:
%%capture
!pip install autoawq
!pip install nvidia-ml-py3

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import notebook_login as hfl
import awq
from pynvml import *
import torch
import time

2024-05-28 10:16:23.091573: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 10:16:23.091670: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 10:16:23.259061: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
hfl()

In [4]:
def printGPUStats():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    
    print(f"GPU Memory Occupied: {info.used // 1024 ** 2} MB.")

In [5]:
%%time
model_id = "0xVolt/Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP-AWQ-4-bit"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map={"": 0})
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

config.json:   0%|          | 0.00/989 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/63.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


This is the memory footprint of the quantized model.

In [6]:
printGPUStats()

GPU Memory Occupied: 5964 MB.


Comapre it to the footprint of Llama-3-8B

In [8]:
if torch.cuda.is_bf16_supported():
    compute_dtype = torch.bfloat16
else:
    compute_dtype = torch.float16
    
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3-8B-Instruct", torch_dtype=compute_dtype, device_map={"": 0})

printGPUStats()

OSError: meta-llama/Llama-3-8B-Instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`